Importando as bibliotecas necessárias

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from ipywidgets import widgets
from functools import reduce
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


Origem do arquivo no Kaggle : https://www.kaggle.com/datasets/middlelight/goodreadsbookswithgenres?resource=download
Fiz o Download e salvei em meu repositorio do Github na pasta do projeto, para manter o histórico, a partir disso será trabalhado tudo em cima do GitHub inclusive o download do arquivo feito por lá. 

In [2]:
url = "https://raw.githubusercontent.com/DanielHanum/Sistemas-Recomendacao-Academico-Final/main/Goodreads_books_with_genres.csv"
base = pd.read_csv(url, encoding='utf-8')


In [3]:
base.head()

,Book Id,Title,Author,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,genres
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Adve..."


verfiicando as colunas do arquivo e entendo mais sobre

In [4]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             11127 non-null  int64  
 1   Title               11127 non-null  object 
 2   Author              11127 non-null  object 
 3   average_rating      11127 non-null  float64
 4   isbn                11127 non-null  object 
 5   isbn13              11127 non-null  int64  
 6   language_code       11127 non-null  object 
 7   num_pages           11127 non-null  int64  
 8   ratings_count       11127 non-null  int64  
 9   text_reviews_count  11127 non-null  int64  
 10  publication_date    11127 non-null  object 
 11  publisher           11127 non-null  object 
 12  genres              11030 non-null  object 
dtypes: float64(1), int64(5), object(7)
memory usage: 1.1+ MB


renomeando as colunas para tornar a visualização mais pratica

In [5]:
# renomeia as colunas
base = base.rename(columns={
    "Id": "COD_LIVRO",
    "Title": "TITULO",
    "Author": "Autor",
    "average_rating": "NOTA_MEDIA",
    "language_code": "IDIOMA",
    "num_pages": "NUM_TOTAL_PAG",
    "ratings_count": "QTD_TOTAL_AVALICAO",
    "text_reviews_count": "QTD_REVIEWS",
    "publication_date": "DATA_PUBLICACAO",
    "publisher": "EDITOR",
    "genres": "GENEROS"
})

# Caso queira salvar o arquivo original em CSV para comparação de originalidade, basta executar o código abaixo
base.to_csv("livros.csv", index=False)

In [6]:
#testando a leitura do arquivo
base1 = pd.read_csv('livros.csv')

In [7]:
base.head()

,Book Id,TITULO,Autor,NOTA_MEDIA,isbn,isbn13,IDIOMA,NUM_TOTAL_PAG,QTD_TOTAL_AVALICAO,QTD_REVIEWS,DATA_PUBLICACAO,EDITOR,GENEROS
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Adve..."


retirando colunas que serão úteis

In [8]:
base = base.drop(columns=["isbn", "isbn13"])

In [9]:
base

,Book Id,TITULO,Autor,NOTA_MEDIA,IDIOMA,NUM_TOTAL_PAG,QTD_TOTAL_AVALICAO,QTD_REVIEWS,DATA_PUBLICACAO,EDITOR,GENEROS
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,eng,352,6333,244,11/1/2003,Scholastic,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,9/13/2004,Scholastic,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Adve..."
...,...,...,...,...,...,...,...,...,...,...,...
11122,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,eng,512,156,20,12/21/2004,Da Capo Press,"Fiction;Writing,Essays;Literature,American;The..."
11123,45633,You Bright and Risen Angels,William T. Vollmann,4.08,eng,635,783,56,12/1/1988,Penguin Books,Fiction;Science Fiction;Literature;Novels;Lite...
11124,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,eng,415,820,95,8/1/1993,Penguin Books,"Historical,Historical Fiction;Fiction;Novels;F..."
11125,45639,Poor People,William T. Vollmann,3.72,eng,434,769,139,2/27/2007,Ecco,"Nonfiction;Sociology;Social Issues,Poverty;His..."


In [10]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             11127 non-null  int64  
 1   TITULO              11127 non-null  object 
 2   Autor               11127 non-null  object 
 3   NOTA_MEDIA          11127 non-null  float64
 4   IDIOMA              11127 non-null  object 
 5   NUM_TOTAL_PAG       11127 non-null  int64  
 6   QTD_TOTAL_AVALICAO  11127 non-null  int64  
 7   QTD_REVIEWS         11127 non-null  int64  
 8   DATA_PUBLICACAO     11127 non-null  object 
 9   EDITOR              11127 non-null  object 
 10  GENEROS             11030 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 956.4+ KB


tirando as linhas que contém branco ou nulo

In [11]:
base = base.dropna()


In [12]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11030 entries, 0 to 11126
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             11030 non-null  int64  
 1   TITULO              11030 non-null  object 
 2   Autor               11030 non-null  object 
 3   NOTA_MEDIA          11030 non-null  float64
 4   IDIOMA              11030 non-null  object 
 5   NUM_TOTAL_PAG       11030 non-null  int64  
 6   QTD_TOTAL_AVALICAO  11030 non-null  int64  
 7   QTD_REVIEWS         11030 non-null  int64  
 8   DATA_PUBLICACAO     11030 non-null  object 
 9   EDITOR              11030 non-null  object 
 10  GENEROS             11030 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 1.0+ MB


editando a coluna de generos para ficar mais fácil a pesquisa

In [13]:
base["GENEROS"] = base["GENEROS"].str.replace(";", "|")
base["GENEROS"] = base["GENEROS"].str.replace(",", "|")

C:\Users\danie\AppData\Local\Temp\ipykernel_17192\2064785856.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base["GENEROS"] = base["GENEROS"].str.replace(";", "|")
C:\Users\danie\AppData\Local\Temp\ipykernel_17192\2064785856.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base["GENEROS"] = base["GENEROS"].str.replace(",", "|")


In [14]:
base.head()

,Book Id,TITULO,Autor,NOTA_MEDIA,IDIOMA,NUM_TOTAL_PAG,QTD_TOTAL_AVALICAO,QTD_REVIEWS,DATA_PUBLICACAO,EDITOR,GENEROS
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,eng,352,6333,244,11/1/2003,Scholastic,Fantasy|Fiction|Young Adult|Fantasy|Magic|Chil...
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,Fantasy|Fiction|Young Adult|Fantasy|Magic|Chil...
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,9/13/2004,Scholastic,Fantasy|Young Adult|Fiction|Fantasy|Magic|Adve...


agora vamos definir o peso de cada avaliação para criterio de escolha do filme
como não temos voto a voto apenas o nome do filme e nota média das avaliações, vamos cortar os livros que não estão inclusos no 3 quartil para baixo das avaliações, ou seja, que tem menos de 25 % da média das avaliações. 
para isso o codigo abaixo:


In [15]:
primeiro_quartil = base["QTD_TOTAL_AVALICAO"].quantile(0.25)

In [16]:
base = base.loc[base["QTD_TOTAL_AVALICAO"] >= primeiro_quartil]

In [17]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8286 entries, 0 to 11126
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             8286 non-null   int64  
 1   TITULO              8286 non-null   object 
 2   Autor               8286 non-null   object 
 3   NOTA_MEDIA          8286 non-null   float64
 4   IDIOMA              8286 non-null   object 
 5   NUM_TOTAL_PAG       8286 non-null   int64  
 6   QTD_TOTAL_AVALICAO  8286 non-null   int64  
 7   QTD_REVIEWS         8286 non-null   int64  
 8   DATA_PUBLICACAO     8286 non-null   object 
 9   EDITOR              8286 non-null   object 
 10  GENEROS             8286 non-null   object 
dtypes: float64(1), int64(4), object(6)
memory usage: 776.8+ KB


In [18]:
base.head()

,Book Id,TITULO,Autor,NOTA_MEDIA,IDIOMA,NUM_TOTAL_PAG,QTD_TOTAL_AVALICAO,QTD_REVIEWS,DATA_PUBLICACAO,EDITOR,GENEROS
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,eng,352,6333,244,11/1/2003,Scholastic,Fantasy|Fiction|Young Adult|Fantasy|Magic|Chil...
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,Fantasy|Fiction|Young Adult|Fantasy|Magic|Chil...
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,9/13/2004,Scholastic,Fantasy|Young Adult|Fiction|Fantasy|Magic|Adve...


In [19]:
base_limpa = base

pronto, agora retirado os livros que na teoria são pouco avaliados, vamos realizar a primeira maneira de recomendação que é a simplificada, utilizando apenas o genero, ordenando a recomendação por maior NOTA_MEDIA depois por QTD_TOTAL_AVALIACAO.

criando a possibilidade do usuario selecionar o perfil dele, criando assim uma adequação as indicações

In [20]:
texto_completo = '|'.join(base['GENEROS'].tolist())

# Divide a string em palavras separadas pelo caractere '|'
palavras = texto_completo.split('|')

# Conta a quantidade de palavras únicas
palavras_unicas = set(palavras)
#Crio um dicionário para calcular a frequencia de repetição das palavras
frequencia_palavras = {}
for palavra in palavras:
    if palavra in frequencia_palavras:
        frequencia_palavras[palavra] += 1
    else:
        frequencia_palavras[palavra] = 1
#Aqui encerro a contagem de cada palavra e identifico quantas palavras unicas existem
total_palavras = len(palavras)
frequencia_relativa = {}
#aqui começo a fazer a conta para descobrir as palavras que tem mais do que 75% da repetição. 
for palavra, frequencia in frequencia_palavras.items():
    frequencia_relativa[palavra] = frequencia / total_palavras
frequencia_relativa_media = sum(frequencia_relativa.values()) / len(frequencia_relativa)
palavras_relevantes = [palavra for palavra, frequencia in frequencia_relativa.items() if frequencia > 0.75 * frequencia_relativa_media]
print("As seguintes palavras aparecem mais do que 75% da frequência relativa média:")
print(palavras_relevantes)

As seguintes palavras aparecem mais do que 75% da frequência relativa média:
['Fantasy', 'Young Adult', 'Fiction', 'Magic', 'Childrens', 'Adventure', 'Audiobook', 'Middle Grade', 'Classics', 'Science Fiction Fantasy', 'Supernatural', 'Mystery', 'Paranormal', 'Novels', 'Science Fiction', 'Humor', 'Comedy', 'European Literature', 'British Literature', 'Nonfiction', 'Science', 'History', 'Historical', 'Philosophy', 'Unfinished', 'Travel', 'Cultural', 'Africa', 'Autobiography', 'Memoir', 'Biography', 'Language', 'Writing', 'Reference', 'Books About Books', 'Essays', 'Literature', 'American', 'The United States Of America', 'Epic Fantasy', 'High Fantasy', 'Art', 'Media Tie In', 'Culture', 'Adult', 'Computer Science', 'Academic', 'School', 'Realistic Fiction', 'Read For School', 'Biography Memoir', 'Romance', 'Anthologies', 'Adult Fiction', 'Paranormal Romance', 'Urban Fantasy', 'Mythology', 'Fairy Tales', 'Juvenile', 'Coming Of Age', 'Historical Fiction', 'Medieval', 'Literary Fiction', 'LG

Apesar de ter filtrado mais do que 75% ainda sobram muitas palavras, mas como já é um numero ok, manterei, mas para diminuir basta aumentar a % no codigo acima

In [21]:
len(palavras_relevantes)

160

In [22]:
opcoes = palavras_relevantes
selecao = widgets.SelectMultiple(options=opcoes, description='Selecione uma opção:(Segure control para selecionar mais do que um)')
display(selecao)

SelectMultiple(description='Selecione uma opção:(Segure control para selecionar mais do que um)', options=('Fa…

transformando o perfil de usuario em uma lista para indicação dos melhores livros

In [23]:
PERFIL_USUARIO = selecao.value
print("Você selecionou os generos:", PERFIL_USUARIO)

Você selecionou os generos: ('Fantasy', 'Young Adult', 'Fiction')


executando script para filtrar somente livros para o usuario de acordo com o perfil dele

In [27]:
mascaras = [base['GENEROS'].apply(lambda x: pref in x) for pref in PERFIL_USUARIO] #aqui aplico a função lambda com for para buscar os generos existentes e relaciona-los ao perfil 
mask = reduce(lambda x, y: x & y, mascaras)#é utilizado a função reduce para aplicar o "AND" ao invez de "OR" afim de ter um resultado mais exato do que o usuário tem relação
recomendados = base[mask]
recomendados = recomendados.sort_values(['NOTA_MEDIA', 'QTD_TOTAL_AVALICAO'], ascending=False)#aqui eu listo primeiro por nota media e como criterio de desempate a qtd de avaliacao


exibindo somente os 10 primeiros

In [28]:
recomendados.head(10)

,Book Id,TITULO,Autor,NOTA_MEDIA,IDIOMA,NUM_TOTAL_PAG,QTD_TOTAL_AVALICAO,QTD_REVIEWS,DATA_PUBLICACAO,EDITOR,GENEROS
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,9/13/2004,Scholastic,Fantasy|Young Adult|Fiction|Fantasy|Magic|Adve...
6,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,eng,3342,28242,808,9/12/2005,Scholastic,Fantasy|Fiction|Young Adult|Fantasy|Magic|Chil...
2528,9325,Fullmetal Alchemist Vol. 10,Hiromu Arakawa/Akira Watanabe,4.60,eng,200,8989,151,11/21/2006,VIZ Media LLC,Sequential Art|Manga|Fantasy|Sequential Art|Co...
6982,26426,Fullmetal Alchemist Vol. 12 (Fullmetal Alchem...,Hiromu Arakawa/Akira Watanabe,4.60,eng,192,7480,119,3/20/2007,VIZ Media LLC,Fantasy|Sequential Art|Graphic Novels|Sequenti...
21,30,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,J.R.R. Tolkien,4.59,eng,1728,101233,1550,9/25/2012,Ballantine Books,Fantasy|Fiction|Classics|Adventure|Science Fic...
6980,26422,Fullmetal Alchemist Vol. 14 (Fullmetal Alchem...,Hiromu Arakawa/Akira Watanabe,4.59,eng,192,8634,118,8/14/2007,VIZ Media LLC,Sequential Art|Manga|Fantasy|Sequential Art|Co...
6233,23506,Fullmetal Alchemist Vol. 11 (Fullmetal Alchem...,Hiromu Arakawa/Akira Watanabe,4.59,eng,192,7655,129,1/16/2007,VIZ Media LLC,Sequential Art|Manga|Fantasy|Sequential Art|Gr...
4273,15336,The Lord of the Rings / The Hobbit,J.R.R. Tolkien,4.59,eng,1600,141,5,10/7/2002,Collins Modern Classics,Fantasy|Fiction|Classics|Adventure|Science Fic...
10922,44734,Fullmetal Alchemist Vol. 6 (Fullmetal Alchemi...,Hiromu Arakawa/Akira Watanabe,4.58,eng,200,10052,201,3/21/2006,VIZ Media LLC,Sequential Art|Manga|Fantasy|Sequential Art|Gr...
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...


In [30]:
recomendados.info()
#veja que apesar de ter uma quantidade maior de livros que atendem as expectativas do usuario, foram lista somente os 10 primeiros.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 647 entries, 4 to 8725
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             647 non-null    int64  
 1   TITULO              647 non-null    object 
 2   Autor               647 non-null    object 
 3   NOTA_MEDIA          647 non-null    float64
 4   IDIOMA              647 non-null    object 
 5   NUM_TOTAL_PAG       647 non-null    int64  
 6   QTD_TOTAL_AVALICAO  647 non-null    int64  
 7   QTD_REVIEWS         647 non-null    int64  
 8   DATA_PUBLICACAO     647 non-null    object 
 9   EDITOR              647 non-null    object 
 10  GENEROS             647 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 60.7+ KB


Agora para começar a misturar, vamos criar uma lista dos livros que o usuario já leu, para evitar que ele assista os mesmo livros, para não dar sempre o mesmo resultado vamos fazer um algoritimo aleatorio, que gere 100 numeros entre 1 e 11127 que é a quantidade de livros originalmente listados no dataset.

In [31]:
HISTORICO = []
for i in range(11120):
    HISTORICO.append(random.randint(1, 11127))

In [32]:
HISTORICO

[260,
 5310,
 8080,
 10471,
 533,
 9513,
 1251,
 1285,
 1452,
 7729,
 3494,
 732,
 9234,
 1242,
 5518,
 10604,
 4252,
 4621,
 8937,
 10596,
 8689,
 6369,
 783,
 7672,
 2578,
 1484,
 6848,
 7081,
 9757,
 5488,
 4228,
 8578,
 9547,
 5671,
 7157,
 9586,
 3355,
 11069,
 3440,
 10223,
 2752,
 8851,
 9922,
 7781,
 3923,
 1303,
 8095,
 1124,
 60,
 5972,
 4251,
 4660,
 10935,
 8318,
 2924,
 2734,
 10457,
 107,
 7255,
 11080,
 8277,
 243,
 6162,
 8966,
 1789,
 4471,
 8581,
 5248,
 2595,
 6406,
 10693,
 5498,
 7574,
 7618,
 2202,
 9537,
 2100,
 5618,
 5435,
 220,
 9957,
 1325,
 328,
 4632,
 594,
 2322,
 9642,
 7703,
 4130,
 2485,
 5954,
 2321,
 5839,
 1157,
 2055,
 6886,
 656,
 7570,
 3729,
 8356,
 6136,
 7612,
 8576,
 2062,
 587,
 1408,
 1170,
 4424,
 5812,
 10951,
 4671,
 10539,
 4305,
 10868,
 2046,
 3117,
 3689,
 3587,
 9065,
 2546,
 1009,
 7113,
 9080,
 9906,
 103,
 1915,
 7058,
 5000,
 8466,
 10241,
 2583,
 10247,
 7017,
 7383,
 562,
 8941,
 1353,
 7664,
 7239,
 897,
 7341,
 822,
 8126,
 7

agora vamos tratar a recomendação dos livros que ele tem relação com o perfil menos os que ele já leu

In [33]:
base['LIDOS'] = np.where(base['Book Id'].isin(HISTORICO), 'Sim', 'Não')

In [34]:
base['LIDOS']

0        Não
1        Não
2        Não
3        Sim
4        Sim
        ... 
11122    Não
11123    Não
11124    Não
11125    Não
11126    Não
Name: LIDOS, Length: 8286, dtype: object

In [59]:
base['LIDOS'].value_counts()

Não    6783
Sim    1503
Name: LIDOS, dtype: int64

In [60]:
base['Book Id']= base['Book Id'].astype(int)
recomendados['Book Id']= recomendados['Book Id'].astype(int)

In [61]:
base_filtrado_lidos = pd.merge(recomendados, base, on="Book Id")

In [62]:
base_filtrado_lidos

,Book Id,TITULO_x,Autor_x,NOTA_MEDIA_x,IDIOMA_x,NUM_TOTAL_PAG_x,QTD_TOTAL_AVALICAO_x,QTD_REVIEWS_x,DATA_PUBLICACAO_x,EDITOR_x,GENEROS_x,TITULO_y,Autor_y,NOTA_MEDIA_y,IDIOMA_y,NUM_TOTAL_PAG_y,QTD_TOTAL_AVALICAO_y,QTD_REVIEWS_y,DATA_PUBLICACAO_y,EDITOR_y,GENEROS_y,LIDOS
0,8,Harry Potter Boxed Set Books 1-5 (Harry Potter #1-5),J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,9/13/2004,Scholastic,Fantasy|Young Adult|Fiction|Fantasy|Magic|Adventure|Fantasy|Supernatural|Mystery|Childrens|Fanta...,Harry Potter Boxed Set Books 1-5 (Harry Potter #1-5),J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,9/13/2004,Scholastic,Fantasy|Young Adult|Fiction|Fantasy|Magic|Adventure|Fantasy|Supernatural|Mystery|Childrens|Fanta...,Sim
1,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,eng,3342,28242,808,9/12/2005,Scholastic,Fantasy|Fiction|Young Adult|Fantasy|Magic|Childrens|Classics|Adventure|Science Fiction Fantasy|N...,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,eng,3342,28242,808,9/12/2005,Scholastic,Fantasy|Fiction|Young Adult|Fantasy|Magic|Childrens|Classics|Adventure|Science Fiction Fantasy|N...,Sim
2,9325,Fullmetal Alchemist Vol. 10,Hiromu Arakawa/Akira Watanabe,4.60,eng,200,8989,151,11/21/2006,VIZ Media LLC,Sequential Art|Manga|Fantasy|Sequential Art|Comics|Sequential Art|Graphic Novels|Fiction|Science...,Fullmetal Alchemist Vol. 10,Hiromu Arakawa/Akira Watanabe,4.60,eng,200,8989,151,11/21/2006,VIZ Media LLC,Sequential Art|Manga|Fantasy|Sequential Art|Comics|Sequential Art|Graphic Novels|Fiction|Science...,Não
3,26426,Fullmetal Alchemist Vol. 12 (Fullmetal Alchemist #12),Hiromu Arakawa/Akira Watanabe,4.60,eng,192,7480,119,3/20/2007,VIZ Media LLC,Fantasy|Sequential Art|Graphic Novels|Sequential Art|Manga|Fiction|Sequential Art|Comics|Science...,Fullmetal Alchemist Vol. 12 (Fullmetal Alchemist #12),Hiromu Arakawa/Akira Watanabe,4.60,eng,192,7480,119,3/20/2007,VIZ Media LLC,Fantasy|Sequential Art|Graphic Novels|Sequential Art|Manga|Fiction|Sequential Art|Comics|Science...,Não
4,30,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings,J.R.R. Tolkien,4.59,eng,1728,101233,1550,9/25/2012,Ballantine Books,Fantasy|Fiction|Classics|Adventure|Science Fiction Fantasy|Fantasy|Epic Fantasy|Fantasy|High Fan...,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings,J.R.R. Tolkien,4.59,eng,1728,101233,1550,9/25/2012,Ballantine Books,Fantasy|Fiction|Classics|Adventure|Science Fiction Fantasy|Fantasy|Epic Fantasy|Fantasy|High Fan...,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,8093,The Stone Fey,Robin McKinley/John Clapp,3.36,eng,64,1036,78,9/1/1998,HMH Books for Young Readers,Fantasy|Young Adult|Fiction|Fantasy|Fairy Tales|Short Stories|Childrens|Childrens|Picture Books|...,The Stone Fey,Robin McKinley/John Clapp,3.36,eng,64,1036,78,9/1/1998,HMH Books for Young Readers,Fantasy|Young Adult|Fiction|Fantasy|Fairy Tales|Short Stories|Childrens|Childrens|Picture Books|...,Sim
643,8486,The Prophet of Yonwood,Jeanne DuPrau,3.27,eng,289,621,82,4/1/2006,Random House Books for Young Readers,Young Adult|Science Fiction|Dystopia|Fantasy|Science Fiction|Fiction|Childrens|Middle Grade|Chil...,The Prophet of Yonwood,Jeanne DuPrau,3.27,eng,289,621,82,4/1/2006,Random House Books for Young Readers,Young Adult|Science Fiction|Dystopia|Fantasy|Science Fiction|Fiction|Childrens|Middle Grade|Chil...,Não
644,2375,The Bone Collector's Son,Paul Yee,3.27,eng,137,117,19,1/1/2004,Two Lions,Young Adult|Historical|Historical Fiction|Fiction|Cultural|Canada|Academic|School|Race|Horror|Fa...,The Bone Collector's Son,Paul Yee,3.27,eng,137,117,19,1/1/2004,Two Lions,Young Adult|Historical|Historical Fiction|Fiction|Cultural|Canada|Academic|School|Race|Horror|Fa...,Sim
645,24775,The Butterfly Tattoo,Philip Pullman,3.19,eng,186,1048,83,9/2/2005,Young Picador,Young Adult|Fiction|Romance|Mystery|Contemporary|Childrens|Fantasy|Thriller|European Literature|...,The Butterfly T

In [63]:
print(base_filtrado_lidos.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 647 entries, 0 to 646
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Book Id               647 non-null    int32  
 1   TITULO_x              647 non-null    object 
 2   Autor_x               647 non-null    object 
 3   NOTA_MEDIA_x          647 non-null    float64
 4   IDIOMA_x              647 non-null    object 
 5   NUM_TOTAL_PAG_x       647 non-null    int64  
 6   QTD_TOTAL_AVALICAO_x  647 non-null    int64  
 7   QTD_REVIEWS_x         647 non-null    int64  
 8   DATA_PUBLICACAO_x     647 non-null    object 
 9   EDITOR_x              647 non-null    object 
 10  GENEROS_x             647 non-null    object 
 11  TITULO_y              647 non-null    object 
 12  Autor_y               647 non-null    object 
 13  NOTA_MEDIA_y          647 non-null    float64
 14  IDIOMA_y              647 non-null    object 
 15  NUM_TOTAL_PAG_y       6

In [64]:
base_resultado = base_filtrado_lidos[base_filtrado_lidos["LIDOS"] == "Não"]


In [65]:
print((base_resultado.loc[:, ['Book Id', 'TITULO_x', 'NOTA_MEDIA_x', 'QTD_TOTAL_AVALICAO_x']]).head(10))


    Book Id  \
2      9325   
3     26426   
4        30   
5     26422   
6     23506   
7     15336   
8     44734   
9         1   
12    15872   
13    26425   

                                                                 TITULO_x  \
2                                            Fullmetal Alchemist  Vol. 10   
3                 Fullmetal Alchemist  Vol. 12 (Fullmetal Alchemist  #12)   
4   J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings   
5                 Fullmetal Alchemist  Vol. 14 (Fullmetal Alchemist  #14)   
6                 Fullmetal Alchemist  Vol. 11 (Fullmetal Alchemist  #11)   
7                                      The Lord of the Rings / The Hobbit   
8                   Fullmetal Alchemist  Vol. 6 (Fullmetal Alchemist  #6)   
9               Harry Potter and the Half-Blood Prince (Harry Potter  #6)   
12             Harry Potter y el misterio del príncipe (Harry Potter  #6)   
13  Fullmetal Alchemist: The Abducted Alchemist (Fullmetal Alche

In [66]:
recomendados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 647 entries, 4 to 8725
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             647 non-null    int32  
 1   TITULO              647 non-null    object 
 2   Autor               647 non-null    object 
 3   NOTA_MEDIA          647 non-null    float64
 4   IDIOMA              647 non-null    object 
 5   NUM_TOTAL_PAG       647 non-null    int64  
 6   QTD_TOTAL_AVALICAO  647 non-null    int64  
 7   QTD_REVIEWS         647 non-null    int64  
 8   DATA_PUBLICACAO     647 non-null    object 
 9   EDITOR              647 non-null    object 
 10  GENEROS             647 non-null    object 
dtypes: float64(1), int32(1), int64(3), object(6)
memory usage: 58.1+ KB


Apesar de não fazer muito "sentido" vou criar uma recomendação também pelo nome do livro lido por último, para isso vou utilizar a coluna criada aleatorio para pegar os últimos lidos. 

In [68]:
ultimo_lido = base.loc[base['LIDOS'] == 'Sim', 'TITULO'].iloc[-1]

In [69]:
#Importando as stopwords em inglês
nltk.download('stopwords')
#importação do modulo de adequação gramatical
nltk.download('punkt')
#importando e testando o modulo de wordnet que é para pegar os sinonimos das palavras
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
title_tokens = word_tokenize(ultimo_lido)
title_words = [lemmatizer.lemmatize(word.lower()) for word in title_tokens if word.lower() not in stop_words]

# Utilizar palavras-chave para recomendação baseada em conteúdo
corpus = recomendados['TITULO']
vectorizer = TfidfVectorizer()
#vetorizando as palavras
X = vectorizer.fit_transform(corpus)
title_vector = vectorizer.transform([' '.join(title_words)])
#calculando similaridade entre cosseno e vetor da palavra lida ( titulo )
cosine_similarities = cosine_similarity(title_vector, X).flatten()
#utilizando o asgort para trazer os 10 primeiros em relação 
related_docs_indices = cosine_similarities.argsort()[:-11:-1]
#trazendo os resultado
result = recomendados.iloc[related_docs_indices]

# ordenando lista de livros recomendados ordenados por relevância
recomendados = result.sort_values(by=['NOTA_MEDIA', 'QTD_TOTAL_AVALICAO'], ascending=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...


In [70]:
recomendados

,Book Id,TITULO,Autor,NOTA_MEDIA,IDIOMA,NUM_TOTAL_PAG,QTD_TOTAL_AVALICAO,QTD_REVIEWS,DATA_PUBLICACAO,EDITOR,GENEROS
4388,15779,Sideways Stories from Wayside School (Wayside School #1),Louis Sachar/Adam McCauley,4.15,eng,144,81090,1835,1/19/2004,Bloomsbury Childrens Books,Childrens|Fiction|Humor|Childrens|Middle Grade|Young Adult|Fantasy|Short Stories|Childrens|Chapt...
3782,13602,ヒカルの碁 16、中国棋院,Yumi Hotta/Yumi Hotta,4.15,jpn,191,282,7,3/4/2002,集英社,Sequential Art|Manga|Sequential Art|Comics|Sequential Art|Graphic Novels|Young Adult|Fiction|Man...
6578,24782,Taran Wanderer (The Chronicles of Prydain #4),Lloyd Alexander,4.14,eng,222,34770,840,5/16/2006,Square Fish,Fantasy|Young Adult|Fiction|Childrens|Adventure|Childrens|Middle Grade|Classics|Science Fiction ...
3922,13986,Arrows of the Queen (Heralds of Valdemar #1),Mercedes Lackey,4.14,eng,320,31058,927,3/3/1987,Daw Books,Fantasy|Fiction|Young Adult|Fantasy|High Fantasy|Science Fiction Fantasy|Fantasy|Magic|Fantasy|E...
1881,6671,The Wonderful Story of Henry Sugar and Six More,Roald Dahl,4.14,eng,240,15116,663,9/11/2001,Knopf Books for Young Readers,Short Stories|Fiction|Childrens|Fantasy|Young Adult|Classics|Childrens|Middle Grade|Humor|Audiob...
7831,30191,Brighid's Quest (Partholon #5),P.C. Cast,4.14,eng,507,3344,107,11/29/2005,Luna Books,Fantasy|Romance|Young Adult|Fantasy|Mythology|Romance|Paranormal Romance|Fantasy|Paranormal|Fant...
10919,44684,Queen of Sorcery (The Belgariad #2),David Eddings,4.14,eng,447,453,20,8/3/2000,Corgi Books (Adult),Fantasy|Fiction|Fantasy|Epic Fantasy|Fantasy|High Fantasy|Young Adult|Science Fiction Fantasy|Ad...
6640,24945,The Wonderful Story Of Henry Sugar and Six More,Roald Dahl,4.14,eng,224,252,21,9/27/2001,Puffin Books,Short Stories|Fiction|Childrens|Fantasy|Young Adult|Classics|Childrens|Middle Grade|Humor|Audiob...
3912,13956,Wise Child,Monica Furlong,4.14,eng,228,178,12,2/24/2004,Random House,Fantasy|Young Adult|Fiction|Historical|Historical Fiction|Childrens|Childrens|Middle Grade|Histo...
5578,20417,Have Space Suit—Will Travel,Robert A. Heinlein,3.90,eng,276,23541,661,2/8/2005,Pocket Books,Science Fiction|Fiction|Young Adult|Science Fiction Fantasy|Classics|Audiobook|Adventure|Space|S...


In [71]:
recomendados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 4388 to 5578
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             10 non-null     int32  
 1   TITULO              10 non-null     object 
 2   Autor               10 non-null     object 
 3   NOTA_MEDIA          10 non-null     float64
 4   IDIOMA              10 non-null     object 
 5   NUM_TOTAL_PAG       10 non-null     int64  
 6   QTD_TOTAL_AVALICAO  10 non-null     int64  
 7   QTD_REVIEWS         10 non-null     int64  
 8   DATA_PUBLICACAO     10 non-null     object 
 9   EDITOR              10 non-null     object 
 10  GENEROS             10 non-null     object 
dtypes: float64(1), int32(1), int64(3), object(6)
memory usage: 920.0+ bytes
